In [ ]:
import os
import sys
import threading
import time

import GCode
import GCode.Line
import GRBL
import matplotlib.pyplot as plt
import numpy as np

# Code:

Draw a 10 mm line from (0, 0) to (10, 0).

In [3]:
progs = list()

for n_points in range(2, 10):

    line_cfg = {"X0": 0, "Xf": 10, "Y": 0, "n_points": n_points}
    points = GCode.HorzLine(**line_cfg)

    line_cfg = {
        "points": points,
        "feed": 120,
        "power": 128,
        "dynamic_power": True,
    }
    line = GCode.Line(points=points)

    prog_cfg = {"lines": [line, line], "feed": 120}
    prog = GCode.Program(**prog_cfg)
    progs.append(prog)
progs


[Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>,
 Program<lines=2, dist=40.00mm, t=14.00s>]

In [5]:
def concurrent_map(func, data):    """    Similar to the bultin function map(). But spawn a thread for each argument    and apply `func` concurrently.    Note: unlike map(), we cannot take an iterable argument. `data` should be an    indexable sequence.    """    N = len(data)    result = [None] * N    # wrapper to dispose the result in the right slot    def task_wrapper(i):        result[i] = func(data[i])    threads = [        threading.Thread(target=task_wrapper, args=(i,)) for i in range(N)    ]    for t in threads:        t.start()    for t in threads:        t.join()    return result

In [6]:
concurrent_map(lambda prog: prog.generate_gcode(), progs)


[None, None, None, None, None, None, None, None]

In [7]:
concurrent_map(lambda prog: len(prog.buffer), progs)


[17, 19, 21, 23, 25, 27, 29, 31]

In [8]:
concurrent_map(lambda prog: prog.__repr__(), progs)


['Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>',
 'Program<lines=2, dist=40.00mm, t=14.00s>']

In [9]:
concurrent_map(lambda prog: prog.dist, progs)


[40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0]

In [10]:
concurrent_map(lambda prog: prog.jog_dist, progs)


[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]

In [11]:
concurrent_map(lambda prog: prog.laserin_dist, progs)


[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]

In [12]:
for prog in progs:
    for line in prog.lines:
        break
    break


In [13]:
list(map(lambda line: line.x_0, prog.lines))


[0.0, 0.0]

In [14]:
list(map(lambda line: line.x_f, prog.lines))


[10.0, 10.0]

In [20]:
prog.jog_dist


20.0

In [21]:
prog.jog_time


10.0

In [22]:
prog.lines[1].reverse()


In [23]:
prog.jog_dist


10.0

In [24]:
prog.jog_time


5.0

In [25]:
prog.lines.reverse()


In [26]:
prog.jog_dist


10.0